In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

In [2]:
# Install Spark and Java
!apt-get update > /dev/null
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

In [3]:
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"


In [4]:
# Start a SparkSession
import findspark
findspark.init()

In [5]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar > /dev/null

--2022-05-01 20:49:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.68MB/s    in 0.6s    

2022-05-01 20:49:06 (1.68 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [15]:
jdbc_url="jdbc:postgresql://dataviz.chvz4idso5az.us-east-1.rds.amazonaws.com/challenge"
df = spark.read.format("jdbc").option("url",jdbc_url).option("dbtable", "vine_table").option("user","").option("password",".").load()


In [16]:
df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RSH1OZ87OYK92|          2|            2|          3|   N|                N|
|R1WFOQ3N9BO65I|          5|            0|          0|   N|                Y|
| R3YOOS71KM5M9|          5|            0|          0|   N|                Y|
|R3R14UATT3OUFU|          5|            0|          0|   N|                Y|
| RV2W9SGDNQA2C|          5|            0|          0|   N|                Y|
|R3CFKLIZ0I2KOB|          5|            0|          0|   N|                Y|
|R1LRYU1V0T3O38|          4|            0|          0|   N|                Y|
| R44QKV6FE5CJ2|          5|            0|          0|   N|                Y|
|R2TX1KLPXXXNYS|          5|            0|          0|   N|                Y|
|R1JEEW4C6R89BA|          5|            0|          0|   N|     

In [17]:
df_filtered = df[df.total_votes > 20]
df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R3EZ0EPYLDA34S|          1|           14|         31|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|     

In [18]:
div_df = df_filtered[(df_filtered.helpful_votes/df_filtered.total_votes) >= 0.5]
print(div_df.count())
div_df.show()

39484
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
| R6KTC1OPIOIIG|          2|           19|         34|   N

In [19]:
vine_program_df = div_df[div_df.verified_purchase == 'Y']
print(vine_program_df.count())
vine_program_df.show()

10223
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2CI0Y288CC7E2|          1|           21|         35|   N|                Y|
|R127WEQY2FM1T3|          1|          147|        175|   N|                Y|
|R1U3AR67RE273L|          1|           51|         65|   N|                Y|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
| R6KTC1OPIOIIG|          2|           19|         34|   N|                Y|
|R1VR5GLGY1GE7N|          1|           49|         51|   N|                Y|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
|R16621F16PQN5A|          3|           24|         31|   N

In [20]:
not_vine_program_df = div_df[div_df.verified_purchase == 'N']
print(not_vine_program_df.count())
not_vine_program_df.show()

29261
+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R4PKAZRQJJX14|          1|           21|         34|   N|                N|
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
|R2FJ94555FZH32|          2|           55|         60|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
|R3PZOXA5X1U8KW|          4|           31|         36|   N|                N|
|R36O341WWXXKNP|          5|           28|         31|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R10LZVBLQHBVJ0|          2|          151|        198|   N|                N|
|R2OWDAR0EDEPTF|          1|           23|         28|   N

In [21]:
five_star_paid = vine_program_df[vine_program_df.star_rating == 5].count()
not_five_star_paid = vine_program_df[vine_program_df.star_rating != 5].count()
percentage = (five_star_paid / not_five_star_paid) * 100
print('Number of 5 stars: ' + str(five_star_paid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 3919 Percentage: 62.17


In [22]:
five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating == 5].count()
not_five_star_unpaid = not_vine_program_df[not_vine_program_df.star_rating != 5].count()
percentage = (five_star_unpaid / not_five_star_unpaid) * 100
print('Number of 5 stars: ' + str(five_star_unpaid), 'Percentage: ' + str(round(percentage,2)))

Number of 5 stars: 11411 Percentage: 63.93
